Este proyecto ha sido realizado por Miguel de la Cal Bravo y Félix Ángel Martínez Muela.
En él, se aplica clustering con series temporales, sobre los datos de hospitalizados en Castilla-La Mancha en cada uno de los hospitales.

In [9]:
# Importamos las librerías
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from math import sqrt
import sys
from scipy import spatial
import numpy as np

In [10]:
# Cargar datos
df = pd.read_csv('../input/hospitalizados/hospitalizados.csv',encoding = "ISO-8859-1", index_col=0, header=None).T  # Lectura del fichero de entrada y transpuesta del mismo
df = df.rename(columns={'Fecha': 'Date'}) # Ajuste del nombre de las columnas
df['Date']= pd.to_datetime(df['Date']) # Conversión a tipo fecha
df.set_index('Date', inplace=True) # La columna Fechas como índice
df=df.apply(pd.to_numeric) # Hacer que los tipos de datos sean numéricos
df = df.sort_values('Date') # Ordenar en función de la fecha
df=df.interpolate('zero', fill_value=0, limit_direction='backward')
df=df.interpolate(method='linear', axis=0).ffill().bfill()
df

,H. Tomelloso,H. Manzanares,H. U. CR,H. Mancha Centro,H. Puertollano,H. Valdepeñas,C. H. Albacete,Almansa,H. Villarrobledo,H. Hellín,H.Toledo,H.N. Parapléjicos,H. Talavera,H. Guadalajara,H. Cuenca
Date,,,,,,,,,,,,,,,
2020-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Los datos cargados los transformamos para tratarlos con mayor facilidad con Time Series Clustering de ahora en adelante.

In [11]:
df_fixed = pd.DataFrame()
col_names = ['date', 'hospital', 'hospitalized']
df_fixed = pd.DataFrame(columns = col_names)

for column in df:
  for index, row in df.iterrows():
    fecha = index
    hospital = column
    hospitalizados = row[column]
    # Añadimos filas al final del nuevo df con la fecha, hospital y hospitalizados
    df_fixed.loc[len(df_fixed)] = [fecha, hospital, hospitalizados]

df_fixed
df_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 0 to 1349
Data columns (total 3 columns):
date            1350 non-null datetime64[ns]
hospital        1350 non-null object
hospitalized    1350 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 42.2+ KB


Después de cargar y dar formato al dataframe, podemos representar la gráfica con los hospitalizados en cada uno de los hospitales de Castilla-La Mancha.

In [18]:
hospitalizados_por_hospital_df = df_fixed.copy()

hospitalizados_por_hospital_df = pd.pivot_table(hospitalizados_por_hospital_df, index='hospital', 
                                       values=['hospitalized', 'date' ], aggfunc=np.mean)
hospitalizados_por_hospital_df['avg_sale'] = hospitalizados_por_hospital_df.apply(lambda r: r.mean(), axis=1)

hospitalizados_hospital_data = go.Scatter(
    y = hospitalizados_por_hospital_df.avg_sale.values,
    mode='markers',
    marker=dict(
        size = hospitalizados_por_hospital_df.avg_sale.values,
        color = hospitalizados_por_hospital_df.avg_sale.values,
        colorscale='Viridis',
        showscale=True
    ),
    text = hospitalizados_por_hospital_df.index.values
)
data = [hospitalizados_hospital_data]

hospitalizados_hospital_layout = go.Layout(
    autosize= True,
    title= 'Gráfica de hospitalizados por hospìtales',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Hospital',
        ticklen= 10,
        zeroline= False,
        gridwidth= 1,
    ),
    yaxis=dict(
        title= 'Nº Hospitalizados',
        ticklen= 10,
        zeroline= False,
        gridwidth= 1,
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=hospitalizados_hospital_layout)
py.iplot(fig,filename='scatter_hospitalizados_hospital')

Aquí ahora podríamos hacer clusters, en función del número de hospitalizados de media durante todo el tiempo

Con los hospitales y hospitalizados por fechas, podemos imprimir la evolución de esta variable en todos los hospitales de Castilla-La Mancha.

In [19]:
def plot_hospital(hospital_id):
    hospital_hospitalized_df = df_fixed.copy()
    multi_hospital_ts_data = []
    for hs in hospital_id:
        flt = hospital_hospitalized_df[hospital_hospitalized_df.hospital == hs]
        multi_hospital_ts_data.append(go.Scatter(x=flt.date, y=flt.hospitalized, name = "H: " + str(hs)))
    py.iplot(multi_hospital_ts_data) 

In [20]:
#df_fixed
df_fixed.hospital.unique()
plot_hospital(df_fixed.hospital.unique())
#plot_hospital(['H. Tomelloso','H. Cuenca']) #Si quisiéramos hacer plot únicamente de los hospitalizados en Tomelloso y Cuenca

En esta gráfica podemos ver que algunos hospitales siguen patrones muy parecidos, lo cual nos puede ayudar de cara al futuro para evitar colapsos en los hospitales. Al observar estos patrones, podríamos también distribuir los pacientes entre hospitales de la comunidad, para así tratarlos con la mayor eficiencia posible.

### Continuar desde aquí el CLUSTERING ###
Now we see each of the sales data react differently to the seasonal pattern. So next I will try to cluster them for each item per store using K means.

In [ ]:
def euclid_dist(t1, t2):
    return np.sqrt(((t1-t2)**2).sum(axis = 1))

In [ ]:
def init_centroids(data, num_clust):
    centroids = np.zeros([num_clust, data.shape[1]]) 
    centroids[0,:] = data[np.random.randint(0, data.shape[0], 1)]

    for i in range(1, num_clust):
        D2 = np.min([np.linalg.norm(data - c, axis = 1)**2 for c in centroids[0:i, :]], axis = 0) 
        probs = D2/D2.sum()
        cumprobs = probs.cumsum()
        ind = np.where(cumprobs >= np.random.random())[0][0]
        centroids[i, :] = np.expand_dims(data[ind], axis = 0)

    return centroids

In [ ]:
def calc_centroids(data, centroids):
    dist = np.zeros([data.shape[0], centroids.shape[0]])

    for idx, centroid in enumerate(centroids):
        dist[:, idx] = euclid_dist(centroid, data)

    return np.array(dist)

In [ ]:
def closest_centroids(data, centroids): 
    dist = calc_centroids(data, centroids) 
    return np.argmin(dist, axis = 1)

In [ ]:
def move_centroids(data, closest, centroids):
    k = centroids.shape[0]
    new_centroids = np.array([data[closest == c].mean(axis = 0) for c in np.unique(closest)])

    if k - new_centroids.shape[0] > 0:
        print("adding {} centroid(s)".format(k - new_centroids.shape[0]))
        additional_centroids = data[np.random.randint(0, data.shape[0], k - new_centroids.shape[0])] 
        new_centroids = np.append(new_centroids, additional_centroids, axis = 0)

    return new_centroids

In [ ]:
def k_means(data, num_clust, num_iter): 
    centroids = init_centroids(data, num_clust)
    last_centroids = centroids

    for n in range(num_iter):
        closest = closest_centroids(data, centroids)
        centroids = move_centroids(data, closest, centroids)
        if not np.any(last_centroids != centroids):
            break
        last_centroids = centroids

    return centroids

In [ ]:
def cosine_similarity(t1, t2):
    return 1 - spatial.distance.cosine(t1, t2)

In [ ]:
store_ids = [s for s in range(1, 11)]
item_ids = [i for i in range(1, 51)]

multi_store_item_df = df.copy()
series_sales = []

for it in item_ids:
#     sales = []
    for st in store_ids:
        flt = multi_store_item_df[multi_store_item_df.store == st]
        flt = flt[flt.item == it]
        series_sales.append(list(flt.sales.values))
    
series_sales = np.reshape(series_sales, (len(series_sales), len(series_sales[0])))
series_sales.shape

In [ ]:
num_cluster = 25
centroids = k_means(series_sales, num_cluster, 100)

sales_clusters = [[] for _ in range(num_cluster)]
for i in range(len(series_sales)):
    clostest_dist = 0
    clust = 0
    for c in range(num_cluster):
        dist = cosine_similarity(centroids[c], series_sales[i])
        if dist > clostest_dist:
            clostest_dist = dist
            clust = c
    sales_clusters[clust].append({
        'store': (i%10) + 1,
        'item': int(i/10) + 1
    })

for sales_cluster in sales_clusters:
    print(len(sales_cluster))

I tried to make 10 cluster and see the number of member on each cluster

In [ ]:
item_ids = []
store_ids = []
for sales_dict in sales_clusters[5]:
    item_ids.append(sales_dict['item'])
    store_ids.append(sales_dict['store'])
plot_sales(item_ids, store_ids)

item_ids = []
store_ids = []    
for sales_dict in sales_clusters[10]:
    item_ids.append(sales_dict['item'])
    store_ids.append(sales_dict['store'])
plot_sales(item_ids, store_ids)

item_ids = []
store_ids = []
for sales_dict in sales_clusters[15]:
    item_ids.append(sales_dict['item'])
    store_ids.append(sales_dict['store'])
plot_sales(item_ids, store_ids)

The member of the same cluster seems to have the very similiar pattern

I think maybe if we use these clusters to train the model, it might help to improve the accuracy. Hope this helps, thank you.